In [2]:
import sys
sys.path.append("../input/tez-lib/")
sys.path.append("../input/timmmaster/")

In [3]:
import tez
import albumentations as A
from albumentations.pytorch import ToTensorV2

import pandas as pd
import numpy as np

import cv2

import timm

import torch
import torch.nn as nn

from sklearn import metrics
from sklearn.model_selection import StratifiedKFold

from tez.callbacks import EarlyStopping
from tqdm import tqdm

In [6]:
class args:
    batch_size = 64
    image_size = 256
    epochs = 20
    fold = 0

In [7]:
class PawpularityDataset:
    def __init__(self, image_paths, dense_features, targets, augmentations):
        self.image_paths = image_paths
        self.dense_features = dense_features
        self.targets = targets
        self.augmentations = augmentations
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, item):
        image = cv2.imread(self.image_paths[item])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.augmentations is not None:
            augmented = self.augmentations(image=image)
            image = augmented['image']
            
        image = np.transpose(image, (2,0,1)).astype(np.float32)
        
        features = self.dense_features[item, :]
        targets = self.targets[item]
        
        return {
            'image':torch.tensor(image, dtype=torch.float),
            'features':torch.tensor(features, dtype=torch.float),
            'targets':torch.tensor(targets, dtype=torch.float)}

In [23]:
class PawpularModel(tez.Model):
    def __init__(self):
        super().__init__()

        self.model = timm.create_model("tf_efficientnet_b0_ns", pretrained=True, in_chans=3)
        self.model.classifier = nn.Linear(self.model.classifier.in_features, 128)
        self.dropout = nn.Dropout(0.1)
        self.out = nn.Linear(128 + 12, 1)  # feature 갯수 12
        
        self.step_scheduler_after = "epoch"

    def monitor_metrics(self, outputs, targets):
        outputs = outputs.cpu().detach().numpy()
        targets = targets.cpu().detach().numpy()
        rmse = metrics.mean_squared_error(targets, outputs, squared=False)
        return {"rmse": rmse}

    def fetch_scheduler(self):
        sch = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
            self.optimizer, T_0=10, T_mult=1, eta_min=1e-6, last_epoch=-1
        )
        return sch

    def fetch_optimizer(self):
        opt = torch.optim.Adam(self.parameters(), lr=1e-4)
        return opt

    def forward(self, image, features, targets=None):

        x = self.model(image)
        x = self.dropout(x)
        x = torch.cat([x, features], dim=1)
        x = self.out(x)

        if targets is not None:
            loss = nn.MSELoss()(x, targets.view(-1, 1))
            metrics = self.monitor_metrics(x, targets)
            return x, loss, metrics
        return x, 0, {}

In [ ]:
train_aug = A.Compose(
    [
        A.Resize(args.image_size, args.image_size, p=1),
        A.HueSaturationValue(
            hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5
        ),
        A.RandomBrightnessContrast(
            brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1), p=0.5
        ),
        A.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0,
            p=1.0,
        ),
    ],
    p=1.0,
)

valid_aug = albumentations.Compose(
    [
        A.Resize(args.image_size, args.image_size, p=1),
        A.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0,
            p=1.0,
        ),
    ],
    p=1.0,
)

In [18]:
BASE_DIR = '../input/petfinder-pawpularity-score'
TRAIN_DIR = '../input/petfinder-pawpularity-score/train'
TEST_DIR = '../input/petfinder-pawpularity-score/test'

df = pd.read_csv(f'{BASE_DIR}/train.csv')

def get_image_path(Id):
    return f'{TRAIN_DIR}/{Id}.jpg'

df['file_path'] = df['Id'].apply(get_image_path)
dense_features = [col for col in df.columns if col not in ['Id', 'Pawpularity', 'file_path']]

In [19]:
# Creat Folds
def create_folds(data, num_splits):
    data["kfold"] = -1
    num_bins = int(np.floor(1 + np.log2(len(data))))

    data.loc[:, "bins"] = pd.cut(data["Pawpularity"], bins=num_bins, labels=False)

    kf = StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=42)
    
    for f, (t_, v_) in enumerate(kf.split(X=data, y=data.bins.values)):
        data.loc[v_, 'kfold'] = f
    
    data = data.drop("bins", axis=1)

    return data

df = create_folds(df, num_splits=10)
df.head()

In [20]:
# 데이터 나누기 (train, valid)
df_train = df[df.kfold != args.fold].reset_index(drop=True)
df_valid = df[df.kfold == args.fold].reset_index(drop=True)

In [21]:
train_dataset = PawpularityDataset(
image_paths=df_train['file_path'].values,
dense_features=df_train[dense_features].values,
targets=df_train.Pawpularity.values,
augmentations=train_aug)

valid_dataset = PawpularityDataset(
image_paths=df_valid['file_path'].values,
dense_features=df_train[dense_features].values,
targets=df_train['Pawpularity'].values,
augmentations=valid_aug)

In [22]:
model = PawpulaModel()

es = EarlyStopping(monitor='valid_rmse',
                   model_path=f'model_f{args.fold}.bin',
                   patience=3,
                   mode='min',
                   save_weights_only=True)

model.fit(
        train_dataset=train_dataset,
        valid_dataset=valid_dataset,
        train_bs=args.batch_size,
        valid_bs=2*args.batch_size,
        device='cuda',
        epochs=args.epochs,
        callbacks=[es],
        fp16=True)